## Baseline GCN testing
Notebook to create and evaluate GCN against EBC on predicting number of passing bicyclists in *copenhagen?*
- Preprocess EBC for graph DONE
- Assign Metrics from data
- Create Torch Graph
- Evaluate against SOTA

In [ ]:
import torch
from torch_geometric.data import Data
import torch_geometric as tg
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from folium import plugins
from folium.plugins import HeatMap
from shapely.geometry import Point, LineString, Polygon
import shapely
import momepy as mp 
import esda
import seaborn as sns
from shapely.strtree import STRtree
import pickle
from tqdm import tqdm

import os, glob, sys

print(os.path.exists('../scripts/bike_functions.py'))
import sys
sys.path.append('../scripts')
from bike_functions import *

sns.set_theme()


In [ ]:
lat, lon = 55.6867243, 12.5700724
dist = 10000

features = [
    'aerialway',
    'aeroway',
    'amenity',
    'barrier',
    'boundary',
    'building',
    'craft',
    'emergency',
    'geological',
    'healthcare',
    'highway',
    'historic',
    'landuse',
    'leisure',
    'man_made',
    'military',
    'natural',
    'office',
    'place',
    'power',
    'public_transport',
    'railway',
    'route',
    'shop',
    'telecom',
    'tourism',
    'water',
    'waterway',
]
features = 'natural,military,man_made,barrier,shop,building,aeroway,landuse,place,railway,historic,power,public_transport,leisure,office,tourism,water,width,telecom,emergency,healthcare,waterway,boundary,craft,route,aerialway,service'.split(',')
expand_features = [
    # 'shop',
    # 'route', 
    # 'highway', 
    # 'waterway', 
    # 'width', 
    # 'length', 
    # 'aerialway', 
    # 'power', 
    # 'healthcare'
]
expand_features = features


In [ ]:
g, gdf, amenities = get_city_graph(lat,
                                    lon,
                                    dist,
                                    features = features, 
                                    expand_features = expand_features)


In [ ]:
H = create_linegraph(g)


### EBC Calculation

In [ ]:
bc = calc_bc(H)
nx.set_node_attributes(H, bc, 'bc')


In [ ]:
gdf_new = load_aadt('../data/raw/trafiktaelling.json', g, gdf)


In [ ]:
H = assign_aadt_to_graph_edges(g, gdf_new, H, aadt_col='aadt_cykler')


In [ ]:
assign_features_to_nodes(H, amenities, geometry_col='geometry', amenity_col='amenity')

In [ ]:
# print the number of nodes in H with the attribute 'aadt'
print(f"Number of nodes in H with 'aadt' attribute: {sum(1 for _, data in H.nodes(data=True) if 'aadt' in data)}")


In [ ]:
for node, value in H.nodes(data=True):
    if 'aadt' not in value.keys():
        value['aadt'] = 0


In [ ]:
all_feats = clean_and_standardize_node_features(H, remove_fields=None)


In [ ]:
### save node features in dataframe
node_features = []
for node in H.nodes(data=True):
    node_features.append(node[1])
node_features = pd.DataFrame(node_features).drop(columns=['aadt', 'osmid'])


In [ ]:
linegraph = graph_to_linegraph_data(H, all_feats, target_feat='aadt', osmid_feat='osmid')


In [ ]:
assert linegraph.edge_index.shape[0] == 2
assert linegraph.edge_index.shape[1] == linegraph.edge_attr.shape[0] if 'edge_attr' in linegraph else True
assert linegraph.x.shape[0] == linegraph.num_nodes


In [ ]:
save_graph_with_config(
    linegraph, 
    H, 
    node_features, 
    features, 
    expand_features, 
    dist
)